In [44]:
import pyaes

#################################STAGE 1###############################################

##### Key round generation

# A 256 bit (32 byte) key
key = "00"* 16
key = "11"* 16
key = bytes.fromhex(key)
print(len(key))


# For some modes of operation we need a random initialization vector
# of 16 bytes
iv = "CAFEBABEFACEDBADDECAF888"
iv = "000000000000000000000000"
iv = bytes.fromhex(iv)

aes = pyaes.AESModeOfOperationECB(key)
plaintext = bytes.fromhex("00"* 16)
H = ciphertext = aes.encrypt(plaintext)

16


In [46]:
#################################STAGE 2 - 3###############################################

##### Cipher text calculation

# X is the plaintext
# ICB is the increased J0
# Key is key
pt = bytes.fromhex('D9313225 F88406E5 A55909C5 AFF5269A 86A7A953 1534F7DA 2E4C303D 8A318A72 1C3C0C95 95680953 2FCF0E24 49A6B525 B16AEDF5 AA0DE657 BA637B39 1AAFD255')
aad = bytes.fromhex('3AD77BB4 0D7A3660 A89ECAF3 2466EF97 F5D3D585 03B9699D E785895A 96FDBAAF 43B1CD7F 598ECE23 881B00E3 ED030688 7B0C785E 27E8AD3F 82232071 04725DD4')
pt = bytes.fromhex('0'* 32)
aad = bytes.fromhex('0'* 32)
# aad = bytes.fromhex('3AD77BB4 0D7A3660 A89ECAF3 2466EF97 F5D3D585 03B9699D E785895A 96FDBAAF 43B1CD7F 598ECE23 881B00E3 ED030688 7B0C785E 27E8AD3F 82232071 04725DD4')



j0 = iv + b'00' * 3 + b'01'
import math
def xor(a, b):
    return bytes([aa^bb for aa, bb in zip(a, b)])
        
def gctr(icb, key, x):
    cb = []
    for i in range(16):
        cb.append(iv + b'\x00' * 3 + bytes([i]))
    if x == "":
        return ''
    y = []
    n = math.ceil(len(x)/16)
    print(cb[2], x)
    for j in range(n):
        cb_ = cb[j + 2]
        ciphed = aes.encrypt(cb_)
#         print(ciphed)
#         print(len(x))
#         for aa, bb in zip(x, ciphed):
#             print('aa ', aa, 'bb ', bb, "aabb ", aa^bb )


        y.append(xor(x[j* 16 : j* 16 + 16], ciphed))
    return y
yy = gctr(j0, key, pt)
C= b''
for y in yy:
    C = C+ y
C.hex()

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02' b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


'f7bd226c6b86ebdb1f6484ef123a2f1c'

In [47]:
#################################STAGE 4 5 6###############################################

##### TAG CALCULATION

# u = aad + C + 3 * b'\x00' + bytes()
def shiftright(a):
    leng = len(a)
    carry = 0
    res = []
    for i in range(leng):
        swap = a[i] % 2
        res.append(a[i] // 2 + carry * (pow(2, 7)))
        carry = swap
    return bytes(res)
        
def gctr1(icb, key, x):
    cb = []
    for i in range(16):
        cb.append(iv + b'\x00' * 3 + bytes([i]))
    if x == "":
        return ''
    y = []
    n = math.ceil(len(x)/16)
#     print(cb[2])
    for j in range(n):
        cb_ = cb[j + 1]
        ciphed = aes.encrypt(cb_)
        y.append(xor(x[j* 16 : j* 16 + 16], ciphed))
    return y

def hashmul(x, y):
    z = b'\x00' * 16 
    v = y
    ele = b'\xe1' + b'\x00' * 15
    
    for i in x:
        num = i
        for j in range(8):
            div = pow(2, 7 - j)
            here = num // div
            num = num % div
            
            if here == 1:
                z = xor(z, v)
            if v[-1] % 2 == 0:
                v = shiftright(v)
            else:
                v = shiftright(v)
                v = xor(v, ele)
    return z
y1 = b'\x00' * 16
# xx = aad + b'\x00' * 48 + C + b'\x00' * 48 + b'\x00' * 5 + b'\x01' + b'\x00' * 2 + b'\x00' * 5 + b'\x01' + b'\x00' * 2 
xx = aad + C + b'\x00' * 7 + b'\x80' + b'\x00' * 7 + b'\x80' 

for i in range(3):
    y1 = hashmul(xor(xx[i * 16:i * 16 +16], y1), H)
S = y1
t = gctr1(j0, key, S)
T = t[0]
T.hex()

'432dad0f7bc3a01eb63416e93bc29235'

In [42]:
print('S is\t\t', S.hex())
print('\nC is\t\t')
for i in range(4):
    print('\t\t', C[i * 16:i * 16 +16].hex())
print('\nTag is','\t\t',T.hex())

S is		 8aacacf4e1ce07662a0eae0a3c9cc30d

C is		
		 0388dace60b6a392f328c2b971b2fe78
		 
		 
		 

Tag is 		 d24e503a1bb037071c71b35d987b8657


In [9]:
    T1 = [ 0xc66363a5, 0xf87c7c84, 0xee777799, 0xf67b7b8d, 0xfff2f20d, 0xd66b6bbd, 0xde6f6fb1, 0x91c5c554, 0x60303050, 0x02010103, 0xce6767a9, 0x562b2b7d, 0xe7fefe19, 0xb5d7d762, 0x4dababe6, 0xec76769a, 0x8fcaca45, 0x1f82829d, 0x89c9c940, 0xfa7d7d87, 0xeffafa15, 0xb25959eb, 0x8e4747c9, 0xfbf0f00b, 0x41adadec, 0xb3d4d467, 0x5fa2a2fd, 0x45afafea, 0x239c9cbf, 0x53a4a4f7, 0xe4727296, 0x9bc0c05b, 0x75b7b7c2, 0xe1fdfd1c, 0x3d9393ae, 0x4c26266a, 0x6c36365a, 0x7e3f3f41, 0xf5f7f702, 0x83cccc4f, 0x6834345c, 0x51a5a5f4, 0xd1e5e534, 0xf9f1f108, 0xe2717193, 0xabd8d873, 0x62313153, 0x2a15153f, 0x0804040c, 0x95c7c752, 0x46232365, 0x9dc3c35e, 0x30181828, 0x379696a1, 0x0a05050f, 0x2f9a9ab5, 0x0e070709, 0x24121236, 0x1b80809b, 0xdfe2e23d, 0xcdebeb26, 0x4e272769, 0x7fb2b2cd, 0xea75759f, 0x1209091b, 0x1d83839e, 0x582c2c74, 0x341a1a2e, 0x361b1b2d, 0xdc6e6eb2, 0xb45a5aee, 0x5ba0a0fb, 0xa45252f6, 0x763b3b4d, 0xb7d6d661, 0x7db3b3ce, 0x5229297b, 0xdde3e33e, 0x5e2f2f71, 0x13848497, 0xa65353f5, 0xb9d1d168, 0x00000000, 0xc1eded2c, 0x40202060, 0xe3fcfc1f, 0x79b1b1c8, 0xb65b5bed, 0xd46a6abe, 0x8dcbcb46, 0x67bebed9, 0x7239394b, 0x944a4ade, 0x984c4cd4, 0xb05858e8, 0x85cfcf4a, 0xbbd0d06b, 0xc5efef2a, 0x4faaaae5, 0xedfbfb16, 0x864343c5, 0x9a4d4dd7, 0x66333355, 0x11858594, 0x8a4545cf, 0xe9f9f910, 0x04020206, 0xfe7f7f81, 0xa05050f0, 0x783c3c44, 0x259f9fba, 0x4ba8a8e3, 0xa25151f3, 0x5da3a3fe, 0x804040c0, 0x058f8f8a, 0x3f9292ad, 0x219d9dbc, 0x70383848, 0xf1f5f504, 0x63bcbcdf, 0x77b6b6c1, 0xafdada75, 0x42212163, 0x20101030, 0xe5ffff1a, 0xfdf3f30e, 0xbfd2d26d, 0x81cdcd4c, 0x180c0c14, 0x26131335, 0xc3ecec2f, 0xbe5f5fe1, 0x359797a2, 0x884444cc, 0x2e171739, 0x93c4c457, 0x55a7a7f2, 0xfc7e7e82, 0x7a3d3d47, 0xc86464ac, 0xba5d5de7, 0x3219192b, 0xe6737395, 0xc06060a0, 0x19818198, 0x9e4f4fd1, 0xa3dcdc7f, 0x44222266, 0x542a2a7e, 0x3b9090ab, 0x0b888883, 0x8c4646ca, 0xc7eeee29, 0x6bb8b8d3, 0x2814143c, 0xa7dede79, 0xbc5e5ee2, 0x160b0b1d, 0xaddbdb76, 0xdbe0e03b, 0x64323256, 0x743a3a4e, 0x140a0a1e, 0x924949db, 0x0c06060a, 0x4824246c, 0xb85c5ce4, 0x9fc2c25d, 0xbdd3d36e, 0x43acacef, 0xc46262a6, 0x399191a8, 0x319595a4, 0xd3e4e437, 0xf279798b, 0xd5e7e732, 0x8bc8c843, 0x6e373759, 0xda6d6db7, 0x018d8d8c, 0xb1d5d564, 0x9c4e4ed2, 0x49a9a9e0, 0xd86c6cb4, 0xac5656fa, 0xf3f4f407, 0xcfeaea25, 0xca6565af, 0xf47a7a8e, 0x47aeaee9, 0x10080818, 0x6fbabad5, 0xf0787888, 0x4a25256f, 0x5c2e2e72, 0x381c1c24, 0x57a6a6f1, 0x73b4b4c7, 0x97c6c651, 0xcbe8e823, 0xa1dddd7c, 0xe874749c, 0x3e1f1f21, 0x964b4bdd, 0x61bdbddc, 0x0d8b8b86, 0x0f8a8a85, 0xe0707090, 0x7c3e3e42, 0x71b5b5c4, 0xcc6666aa, 0x904848d8, 0x06030305, 0xf7f6f601, 0x1c0e0e12, 0xc26161a3, 0x6a35355f, 0xae5757f9, 0x69b9b9d0, 0x17868691, 0x99c1c158, 0x3a1d1d27, 0x279e9eb9, 0xd9e1e138, 0xebf8f813, 0x2b9898b3, 0x22111133, 0xd26969bb, 0xa9d9d970, 0x078e8e89, 0x339494a7, 0x2d9b9bb6, 0x3c1e1e22, 0x15878792, 0xc9e9e920, 0x87cece49, 0xaa5555ff, 0x50282878, 0xa5dfdf7a, 0x038c8c8f, 0x59a1a1f8, 0x09898980, 0x1a0d0d17, 0x65bfbfda, 0xd7e6e631, 0x844242c6, 0xd06868b8, 0x824141c3, 0x299999b0, 0x5a2d2d77, 0x1e0f0f11, 0x7bb0b0cb, 0xa85454fc, 0x6dbbbbd6, 0x2c16163a ]
    

In [48]:
qqq= [0]* 15
qqq.append(2)
aes.encrypt(bytes(qqq))
len(xx)

48

In [5]:
C

b"\x8a\xa3=\xf6\xc8\xe2c\xd0\xff:9c:4\xe3\x17t\x1c\x99m('B+\n\x9c_\x0f\xb2\x8c:M,\xf8\t6\xd0\x1f\x8b^CQuo\xf43\x0e1\xbb\xd8\xf7\xdd\xd1\x16[\xd1\x96*\x8a\xb9L\x81\x9a\xc1"

In [19]:
'0'* 32

'00000000000000000000000000000000'

In [30]:
len(bytes.fromhex(([0]* 15).append(2)))

TypeError: fromhex() argument must be str, not None

In [26]:
[0 * 15, 0]

[0, 0]

In [29]:
([0]* 15).append(2)

AttributeError: 'AESModeOfOperationECB' object has no attribute '_Ke'

In [18]:
key = "ff"* 16
import pyaes
key = bytes.fromhex(key)
aaa = pyaes.AES(key)

In [25]:
aaa._Ke
import numpy as np
np.uint(aaa._Ke[1][0])%(pow(2, 24)) //(pow(2, 16))
# aaa._Ke[1][0]%(pow(2, 24))//(pow(2, 16))

233.0

In [26]:
bytes([233])

b'\xe9'

In [88]:
import struct
tk = [ struct.unpack('>i', key[i:i + 4])[0] for i in range(0, len(key), 4) ]

In [16]:
aaa._Ke

[[286331153, 286331153, 286331153, 286331153],
 [2459145107, 2206368386, 2459145107, 2206368386],
 [2206236799, 131837, 2459013486, 286462956],
 [4210904829, 4211035136, 1752096110, 2038288002],
 [225860939, 4152955211, 2682768421, 3868852903],
 [2768306629, 1384648846, 3446660267, 737533452],
 [1662844724, 831844282, 4244321041, 3608079645],
 [2286508858, 3118314624, 1160152977, 2452122252],
 [1011564405, 2241235957, 3232762980, 1385719528],
 [1635296373, 3840814976, 610149348, 1992705292],
 [3809892941, 133723597, 598103593, 1432359717]]

In [104]:
hashmul([0]*7 +[10], [0]*7 +[20]).hex()

'38400000000001de'

In [97]:
a

b'\xe6\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03'

In [102]:
a

b'8@\x00\x00\x00\x00\x01\xde'

In [30]:
pt = bytes.fromhex('D9313225 F88406E5 A55909C5 AFF5269A 86A7A953 1534F7DA 2E4C303D 8A318A72 1C3C0C95 95680953 2FCF0E24 49A6B525 B16AEDF5 AA0DE657 BA637B39 1AAFD255')
aad = bytes.fromhex('3AD77BB4 0D7A3660 A89ECAF3 2466EF97 F5D3D585 03B9699D E785895A 96FDBAAF 43B1CD7F 598ECE23 881B00E3 ED030688 7B0C785E 27E8AD3F 82232071 04725DD4')
pt = bytes.fromhex('0'* 32)
aad = bytes.fromhex('0'* 32)
# aad = bytes.fromhex('3AD77BB4 0D7A3660 A89ECAF3 2466EF97 F5D3D585 03B9699D E785895A 96FDBAAF 43B1CD7F 598ECE23 881B00E3 ED030688 7B0C785E 27E8AD3F 82232071 04725DD4')



j0 = iv + b'00' * 3 + b'01'
import math
def xor(a, b):
    return bytes([aa^bb for aa, bb in zip(a, b)])
        
def gctr(icb, key, x):
    cb = []
    for i in range(16):
        cb.append(iv + b'\x00' * 3 + bytes([i+1]))
    if x == "":
        return ''
    y = []
    n = math.ceil(len(x)/16)
    print(cb[2], x)
    for j in range(n):
        cb_ = cb[j + 2]
        ciphed = aes.encrypt(cb_)
        print(ciphed)
#         print(len(x))

        for aa, bb in zip(x, ciphed):
            print('aa ', aa, 'bb ', bb, "aabb ", aa^bb )


        y.append(xor(x[j* 16 : j* 16 + 16], ciphed))
    return y
# print()    
yy = gctr(j0, key, pt)
C= b''
for y in yy:
    C = C+ y
C.hex()

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03' b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xf7\x95\xaa\xabIKY#\xf7\xfd\x89\xff\x94\x8b\xc1\xe0'
aa  0 bb  247 aabb  247
aa  0 bb  149 aabb  149
aa  0 bb  170 aabb  170
aa  0 bb  171 aabb  171
aa  0 bb  73 aabb  73
aa  0 bb  75 aabb  75
aa  0 bb  89 aabb  89
aa  0 bb  35 aabb  35
aa  0 bb  247 aabb  247
aa  0 bb  253 aabb  253
aa  0 bb  137 aabb  137
aa  0 bb  255 aabb  255
aa  0 bb  148 aabb  148
aa  0 bb  139 aabb  139
aa  0 bb  193 aabb  193
aa  0 bb  224 aabb  224


'f795aaab494b5923f7fd89ff948bc1e0'